In [5]:
from web3 import Web3
from eth_account import Account

# ---------------- STEP 1: CREATE A WALLET ----------------
wallet = Account.create()
WALLET_ADDRESS = wallet.address
PRIVATE_KEY = wallet.key.hex()  # Store securely!

print(f"✅ Wallet Created!\nAddress: {WALLET_ADDRESS}\nPrivate Key: {PRIVATE_KEY}")

✅ Wallet Created!
Address: 0xCf3Bf20DDB7BA891a81a74d890D776E4EEECFb74
Private Key: 5b51da12899cc7d65974064e7dfdc15cce4a62ab78bba4fc0dbc0e0492fc4036


In [93]:

# ---------------- STEP 2: CONNECT TO GENSYN TESTNET ----------------
ALCHEMY_URL = "https://gensyn-testnet.g.alchemy.com/public"
web3 = Web3(Web3.HTTPProvider(ALCHEMY_URL))

if web3.is_connected():
    print("✅ Connected to Gensyn Testnet")
else:
    raise Exception("Failed to connect to Gensyn Testnet")


✅ Connected to Gensyn Testnet


In [94]:

# ---------------- STEP 3: CHECK BALANCE ----------------
balance = web3.eth.get_balance(WALLET_ADDRESS)
eth_balance = web3.from_wei(balance, "ether")
print(f"💰 Wallet Balance: {eth_balance} ETH")


💰 Wallet Balance: 0.034893640919914966 ETH


In [95]:
# ---------------- STEP 4: CONTRACT CONFIG ----------------

with open('../out/SwarmCoordinator.sol/SwarmCoordinator.json', 'r') as f:
   contract_abi = json.load(f)['abi']

contract_address = "0x915674138096b84aa3CD05cB0F9c2EEE24b192C5"
contract = web3.eth.contract(address=contract_address, abi=contract_abi)


In [96]:
# ---------------- STEP 5: CURRENT ROUND, STAGE ----------------

# Get the current round and stage
current_round = contract.functions.currentRound().call()
current_stage = contract.functions.currentStage().call()

print(f"Current Round: {current_round}")
print(f"Current Stage: {current_stage}")

Current Round: 0
Current Stage: 0


In [97]:
# ---------------- STEP 6: REGISTER PEER ----------------

# Peer ID and private key
peer_id = b"grieve_peer"

checksummed = Web3.to_checksum_address(WALLET_ADDRESS)
nonce = web3.eth.get_transaction_count(checksummed)


# Build the transaction
txn = contract.functions.registerPeer(peer_id).build_transaction({
    'chainId': 685685,  # Mainnet
    'gas': 500000,
    'gasPrice': web3.to_wei('50', 'gwei'),
    'nonce': nonce,
})

# Sign the transaction
signed_txn = web3.eth.account.sign_transaction(txn, private_key=PRIVATE_KEY)

# Send the transaction
tx_hash = web3.eth.send_raw_transaction(signed_txn.raw_transaction)

# Print the transaction hash
print(f"Transaction hash: {web3.to_hex(tx_hash)}")

Transaction hash: 0x92bf612c04f6eda428350afd5677458d929bca2d3bc01a508f6af5f51c261f6f
